In [3]:
! uv pip install langchain openai tiktoken rapidocr-onnxruntime python-dotenv langchain-community

Using Python 3.11.5 environment at: /Users/yashchinawale/Desktop/end-end-rag/.venv
Audited 6 packages in 3ms


In [3]:
import os

from dotenv import load_dotenv

load_dotenv()


os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


## Data Ingestion


In [4]:
from langchain_community.document_loaders import TextLoader

In [5]:
loader = TextLoader("/Users/yashchinawale/Desktop/end-end-rag/data/AgenticAi.txt", encoding="utf8")
documents = loader.load()

In [6]:
documents[0].page_content[:500]  # Print the first 500 characters of the first documen

'Agentic AI refers to an advanced paradigm of artificial intelligence where systems go beyond merely responding to queries or performing isolated tasks. Unlike traditional AI, which operates under predefined rules and focuses on specific tasks, agentic AI systems exhibit autonomy by actively planning, reasoning, executing complex actions, and adapting to dynamic environments with minimal human intervention.\n\nKey Characteristics of Agentic AI\nAutonomy and Proactiveness: Agentic AI operates indepen'

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [8]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=20)

In [9]:
text_chunks=text_splitter.split_documents(documents)

In [10]:
text_chunks

[Document(metadata={'source': '/Users/yashchinawale/Desktop/end-end-rag/data/AgenticAi.txt'}, page_content='Agentic AI refers to an advanced paradigm of artificial intelligence where systems go beyond merely responding to queries or performing isolated tasks. Unlike traditional AI, which operates under'),
 Document(metadata={'source': '/Users/yashchinawale/Desktop/end-end-rag/data/AgenticAi.txt'}, page_content='operates under predefined rules and focuses on specific tasks, agentic AI systems exhibit autonomy by actively planning, reasoning, executing complex actions, and adapting to dynamic environments'),
 Document(metadata={'source': '/Users/yashchinawale/Desktop/end-end-rag/data/AgenticAi.txt'}, page_content='environments with minimal human intervention.'),
 Document(metadata={'source': '/Users/yashchinawale/Desktop/end-end-rag/data/AgenticAi.txt'}, page_content='Key Characteristics of Agentic AI'),
 Document(metadata={'source': '/Users/yashchinawale/Desktop/end-end-rag/data/Agentic

In [15]:
! uv pip install faiss-cpu


Using Python 3.11.5 environment at: /Users/yashchinawale/Desktop/end-end-rag/.venv
Resolved 3 packages in 404ms                                         
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)-------------------     0 B/3.20 MiB            
⠙ Preparing packages... (0/1)------------------- 16.00 KiB/3.20 MiB          
⠙ Preparing packages... (0/1)------------------- 32.00 KiB/3.20 MiB          
⠙ Preparing packages... (0/1)------------------- 48.00 KiB/3.20 MiB          
⠙ Preparing packages... (0/1)------------------- 64.00 KiB/3.20 MiB          
⠙ Preparing packages... (0/1)------------------- 80.00 KiB/3.20 MiB          
⠙ Preparing packages... (0/1)------------------- 96.00 KiB/3.20 MiB          
⠙ Preparing packages... (0/1)------------------- 112.00 KiB/3.20 MiB         
⠙ Preparing packages... (0/1)------------------- 128.00 KiB/3.20 MiB         
⠙ Preparing packages... (0/1)------------------- 144.00 KiB/3.20

In [18]:
! uv pip install langchain-openai langchain-community

Using Python 3.11.5 environment at: /Users/yashchinawale/Desktop/end-end-rag/.venv
Resolved 52 packages in 375ms                                        
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)-------------------     0 B/80.01 KiB           
⠙ Preparing packages... (0/1)------------------- 16.00 KiB/80.01 KiB         
⠙ Preparing packages... (0/1)------------------- 32.00 KiB/80.01 KiB         
⠙ Preparing packages... (0/1)0m------------- 48.00 KiB/80.01 KiB         
⠙ Preparing packages... (0/1)---------- 64.00 KiB/80.01 KiB         
⠙ Preparing packages... (0/1)---------- 80.00 KiB/80.01 KiB         
⠙ Preparing packages... (0/1)---------- 80.01 KiB/80.01 KiB         
Prepared 1 package in 118ms                                                       
Installed 1 package in 3ms.0.2                              
 + langchain-openai==1.0.2


In [11]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS

In [12]:
embeddings=OpenAIEmbeddings()

In [13]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

In [14]:
vectorstore

In [15]:
retriever=vectorstore.as_retriever()

In [16]:
# Perform similarity search
query = "What is the Key Characteristics of Agentic AI?"
docs = vectorstore.similarity_search(query, k=4)

# Display the results
for i, doc in enumerate(docs):
    print(f"Document {i+1}:")
    print(doc.page_content)
    print("-" * 50)


Document 1:
Key Characteristics of Agentic AI
--------------------------------------------------
Document 2:
Agentic AI vs. Traditional AI
Feature	Traditional AI	Agentic AI
Autonomy	Low, responds to input only	High, acts independently and proactively
--------------------------------------------------
Document 3:
operates under predefined rules and focuses on specific tasks, agentic AI systems exhibit autonomy by actively planning, reasoning, executing complex actions, and adapting to dynamic environments
--------------------------------------------------
Document 4:
Autonomy and Proactiveness: Agentic AI operates independently, initiating and managing actions toward long-term goals rather than waiting for explicit instructions, unlike traditional AI that reacts
--------------------------------------------------


In [17]:
from langchain_core.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [18]:
prompt=ChatPromptTemplate.from_template(template)

In [19]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [20]:
from langchain_core.output_parsers import StrOutputParser


In [21]:
output_parser=StrOutputParser()

In [23]:
from langchain_openai import ChatOpenAI

llm_model=ChatOpenAI(model_name="gpt-4o-mini")

In [24]:
from langchain_core.runnables import RunnablePassthrough


rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

In [25]:
rag_chain.invoke("tell me about Agentic AI")

'Agentic AI represents an advanced paradigm of artificial intelligence that goes beyond simply answering queries or performing isolated tasks. Unlike traditional AI, which follows predefined rules and concentrates on specific tasks, Agentic AI systems operate with a degree of autonomy. They are capable of actively planning, reasoning, and executing complex actions. These systems can adapt to dynamic environments and refine their strategies based on experience, which allows for more sophisticated decision-making. This level of sophistication enables Agentic AI to exhibit advanced reasoning that is not merely rule-based or pattern-based. In summary, Agentic AI is characterized by its ability to operate independently and learn from interactions, making it a powerful tool in various applications.'